In [33]:
import numpy as np

# -------------------------
# Block encoding (Halmos dilation)
# -------------------------

def qsp_U00_batch(xs, phases):
    W_list = [W_signal(x) for x in xs]
    res = []
    for W in W_list:
        U = Rz(phases[0])
        for phi in phases[1:]:
            U = W @ U
            U = Rz(phi) @ U
        res.append(U[0, 0])
    return np.array(res)

def psd_sqrt(M):
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def block_encode_halmos(A, alpha=None):
    """
    Returns (alpha, U) where U is 2n x 2n unitary and top-left block is A/alpha.
    A need not be Hermitian/unitary.
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square for this simple Halmos block-encoding.")
    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0

    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])
    return alpha, U


# -------------------------
# QSP core (2x2 signal)
# -------------------------

def W_signal(x):
    """2x2 signal unitary for QSP; x in [-1,1]."""
    x = float(x)
    if abs(x) > 1 + 1e-12:
        raise ValueError("x must be in [-1,1]")
    s = np.sqrt(max(0.0, 1.0 - x*x))
    return np.array([[x, 1j*s],
                     [1j*s, x]], dtype=np.complex128)

def Rz(phi):
    return np.diag([np.exp(1j*phi), np.exp(-1j*phi)]).astype(np.complex128)

def qsp_U00(x, phases):
    """
    U(x) = Rz(phi0) W(x) Rz(phi1) W(x) ... W(x) Rz(phid)
    return (0,0) entry, which is the QSP polynomial value (complex in general).
    """
    U = Rz(phases[0])
    W = W_signal(x)
    for phi in phases[1:]:
        U = W @ U
        U = Rz(phi) @ U
    return U[0, 0]


# -------------------------
# Target function: truncated inverse on [-1,1]
# -------------------------

def truncated_inverse_target(x, kappa, x0=None, x1=None):
    """
    Odd extension target for QSP:
      - for |x| >= x1: g(x) = 1/(kappa*x)
      - for |x| <= x0: g(x) = 0
      - smooth cubic transition in between (C1-ish, bounded)

    Default: x1 = 1/kappa, x0 = 0.5/kappa
    (so we don't force behavior extremely near 0 where inverse is nasty)
    """
    x = float(x)
    sgn = 1.0 if x >= 0 else -1.0
    ax = abs(x)
    if x1 is None: x1 = 1.0 / kappa
    if x0 is None: x0 = 0.5 / kappa
    if x0 <= 0 or x1 <= x0:
        raise ValueError("Need 0 < x0 < x1.")
    if ax <= x0:
        return 0.0
    if ax >= x1:
        return sgn * (1.0 / (kappa * ax))

    # Smoothstep t in [0,1]
    t = (ax - x0) / (x1 - x0)
    smooth = t*t*(3 - 2*t)  # cubic smoothstep

    # Blend from 0 to 1/(kappa*ax); keep bounded by <= 1
    return sgn * smooth * (1.0 / (kappa * ax))

def clipped_inverse_target(x, kappa, x_min=None):
    """
    일부 방향을 0으로 버리지 않고, 작은 x에서는 inverse 값을 saturation 시켜서 사용.
    g(x) ~ 1/(kappa * x), 단 |g(x)| <= 1 이 되도록 잘라줌.
    """
    x = float(x)
    if x == 0.0:
        # 정확히 0은 피하기
        x = 1e-12
    sgn = 1.0 if x >= 0 else -1.0
    ax = abs(x)

    # 역수를 정의할 때 사용할 최소 x 크기
    if x_min is None:
        # 예: 이건 설계 선택 (추측입니다)
        x_min = 1.0 / kappa   # 혹은 더 작게 잡아도 됨

    # 너무 작으면 x_min으로 클리핑
    ax_eff = max(ax, x_min)

    val = 1.0 / (kappa * ax_eff)   # inverse 값
    # QSP용으로 |g(x)| <= 1 유지하려면 1로 saturation
    if val > 1.0:
        val = 1.0

    return sgn * val


# -------------------------
# Phase fitter (robust-ish)
# -------------------------

class QSPPhaseFitter:
    """
    Fits phases to approximate a target function g(x) on [-1,1] (or subset),
    with penalties to enforce |p(x)| <= 1 (required for QSP feasibility).
    """

    def __init__(self, degree, kappa, seed=0):
        self.degree = int(degree)
        self.kappa = float(kappa)
        self.rng = np.random.default_rng(seed)

    def _make_grids(self, n_cheb=128, n_dense=1024, focus_min=None):
        # Chebyshev nodes on [-1,1]
        j = np.arange(n_cheb)
        x_cheb = np.cos((2*j + 1) * np.pi / (2*n_cheb))

        # Dense uniform grid for safety checks
        x_dense = np.linspace(-1.0, 1.0, n_dense)

        if focus_min is None:
            # Optionally focus loss on |x| >= focus_min (e.g. 1/kappa zone)
            focus_min = 1.0 / self.kappa
        x_cheb = x_cheb[np.abs(x_cheb) >= focus_min]
        x_dense = x_dense[np.abs(x_dense) >= focus_min]

        return x_cheb, x_dense

    def fit(self,
            target_fn,
            n_cheb=16,
            n_dense=16,
            focus_min=None,
            iters=10,
            lr=0.05,
            penalty_w=50.0,
            bound_eps=1e-6,
            multi_start=6):
        """
        Returns best_phases, info dict.
        - penalty enforces |p(x)|<=1 on dense grid (soft).
        - multi_start tries multiple random initializations and picks the best verified.
        """
        d = self.degree
        best = None
        best_info = None

        x_cheb, x_dense = self._make_grids(n_cheb=n_cheb, n_dense=n_dense, focus_min=focus_min)
        y_cheb = np.array([target_fn(x) for x in x_cheb], dtype=np.float64)
        y_dense = np.array([target_fn(x) for x in x_dense], dtype=np.float64)

        def loss(phases):
            # Primary fit on Chebyshev nodes
            vals = qsp_U00_batch(x_cheb, phases)
            # We want real target; take real part but also penalize imag leakage
            diff_re = (vals.real - y_cheb)
            diff_im = vals.imag
            mse = np.mean(diff_re*diff_re) + 0.2*np.mean(diff_im*diff_im)

            # Bound penalty on dense grid
            vd = qsp_U00_batch(x_dense, phases)
            abs_v = np.abs(vd)
            viol = np.clip(abs_v - (1.0 + bound_eps), 0.0, None)
            pen = penalty_w * np.mean(viol*viol)

            return mse + pen

        def finite_diff_grad(phases, eps=1e-4):
            g = np.zeros_like(phases)
            base = loss(phases)
            for i in range(len(phases)):
                p1 = phases.copy(); p1[i] += eps
                p2 = phases.copy(); p2[i] -= eps
                g[i] = (loss(p1) - loss(p2)) / (2*eps)
            return base, g

        def optimize_from(init):
            phases = init.copy()
            # simple Adam + backtracking-ish
            m = np.zeros_like(phases)
            v = np.zeros_like(phases)
            b1, b2 = 0.9, 0.999
            eps_adam = 1e-8

            cur = loss(phases)
            for t in range(1, iters+1):
                cur, g = finite_diff_grad(phases)
                m = b1*m + (1-b1)*g
                v = b2*v + (1-b2)*(g*g)
                mhat = m / (1 - b1**t)
                vhat = v / (1 - b2**t)
                step = lr * mhat / (np.sqrt(vhat) + eps_adam)

                # backtracking to avoid explosions
                new_phases = phases - step
                new_loss = loss(new_phases)
                if new_loss <= cur:
                    phases = new_phases
                    cur = new_loss
                else:
                    # shrink step
                    phases = phases - 0.3*step
                    cur = loss(phases)

                # keep phases in a reasonable range (wrap)
                phases = (phases + np.pi) % (2*np.pi) - np.pi

            return phases, cur

        def verify(phases):
            # Check max |p(x)| on full dense [-1,1]
            xs = np.linspace(-1.0, 1.0, 2048)
            vals = np.array([qsp_U00(x, phases) for x in xs])
            max_abs = float(np.max(np.abs(vals)))
            # Error on inverse region |x| >= 1/kappa
            mask = np.abs(xs) >= (1.0 / self.kappa)
            err = float(np.max(np.abs(vals.real[mask] - np.array([target_fn(x) for x in xs[mask]]))))
            imag_max = float(np.max(np.abs(vals.imag)))
            return max_abs, err, imag_max

        for s in range(multi_start):
            init = self.rng.normal(scale=0.2, size=d+1)
            phases, L = optimize_from(init)
            max_abs, err, imag_max = verify(phases)

            # Prefer feasible-ish and low error
            score = L + 10.0*max(0.0, max_abs-1.0) + err + 0.1*imag_max
            info = {"loss": L, "max_abs": max_abs, "max_err_on_[1/k,1]": err, "max_imag": imag_max}

            if best is None or score < best_info["score"]:
                best = phases
                best_info = {"score": score, **info}

        return best, best_info


# -------------------------
# Reliable-ish solver: build block encoding + fit QSP phases + apply via SVD (trustworthy SVT action)
# -------------------------

class ReliableQSVTInverseApprox:
    """
    - Halmos block encoding으로 U_block (2n x 2n) 생성
    - QSP로 inverse용 다항식 p(x) 피팅
    - p(x)를 block-encoded A에 QSVT 방식으로 적용해서
      top-left 블록으로 A^{-1} 근사를 만드는 시뮬레이터
    """

    def __init__(self, A, kappa, degree=9, seed=0):
        self.A = np.array(A, dtype=np.complex128)
        n, m = self.A.shape
        if n != m:
            raise ValueError("This minimal solver assumes square A.")
        self.n = n
        self.kappa = float(kappa)
        if self.kappa < 1:
            raise ValueError("kappa must be >= 1")

        # Block encoding: U_block은 2n x 2n 유니터리, top-left가 A/alpha
        self.alpha, self.U_block = block_encode_halmos(self.A)

        # QSP phase fitter (스칼라용)
        self.fitter = QSPPhaseFitter(degree=degree, kappa=self.kappa, seed=seed)

        # target 함수: clipped inverse
        self.target_fn = lambda x: clipped_inverse_target(x, self.kappa)

        self.phases = None
        self.fit_info = None

        # 행렬 레벨에서 쓸 Z, W_A 준비용 캐시
        self._Z_big = None
        self._W_A = None

    # -------------------------
    # Phase fitting (스칼라 QSP)
    # -------------------------
    def fit_phases(self,
                   n_cheb=64,
                   n_dense=128,
                   iters=600,
                   lr=0.05,
                   penalty_w=80.0,
                   multi_start=8):
        phases, info = self.fitter.fit(
            target_fn=self.target_fn,
            n_cheb=n_cheb,
            n_dense=n_dense,
            focus_min=None,
            iters=iters,
            lr=lr,
            penalty_w=penalty_w,
            multi_start=multi_start
        )
        self.phases = phases
        self.fit_info = info
        return phases, info

    def p(self, x):
        """QSP polynomial value (scalar)."""
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")
        return qsp_U00(x, self.phases)

    # -------------------------
    # 행렬 레벨 QSVT용 빌더들
    # -------------------------
    def _build_Z_big(self):
        """
        Z_big = 2P - I, where P = diag(I_n, 0_n).
        즉, diag(I_n, -I_n) (2n x 2n).
        """
        if self._Z_big is not None:
            return self._Z_big
        n = self.n
        Z = np.zeros((2*n, 2*n), dtype=np.complex128)
        Z[:n, :n] = np.eye(n, dtype=np.complex128)
        Z[n:, n:] = -np.eye(n, dtype=np.complex128)
        self._Z_big = Z
        return Z

    def _build_W_A(self):
        """
        행렬 레벨 signal unitary:
            W_A = Z_big @ U_block
        QSVT 이론에서 (2P - I) U 가 scalar QSP의 W(x)에 해당.
        """
        if self._W_A is not None:
            return self._W_A
        Z = self._build_Z_big()
        self._W_A = Z @ self.U_block
        return self._W_A

    def _R_big(self, phi):
        """
        R_Z(phi) = exp(i * phi * Z_big)
                  = blockdiag(e^{i phi} I_n, e^{-i phi} I_n)
        """
        n = self.n
        R = np.zeros((2*n, 2*n), dtype=np.complex128)
        e_p = np.exp(1j * phi)
        e_m = np.exp(-1j * phi)
        R[:n, :n] = e_p * np.eye(n, dtype=np.complex128)
        R[n:, n:] = e_m * np.eye(n, dtype=np.complex128)
        return R

    def _build_qsvt_unitary(self):
        """
        스칼라 QSP 구조를 그대로 2n x 2n에 올린 QSVT 유니터리:
            U = R(phi_0)
            for phi in phases[1:]:
                U = W_A @ U
                U = R(phi) @ U
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")
        phases = self.phases
        W_A = self._build_W_A()

        # 초기 R(phi_0)
        U = self._R_big(phases[0])

        # 나머지 phase들 적용
        for phi in phases[1:]:
            U = W_A @ U
            U = self._R_big(phi) @ U

        return U

    # -------------------------
    # QSVT로 A^{-1} 근사 적용
    # -------------------------
    def apply_inverse_approx(self, b):
        """
        QSVT 유니터리에서 top-left 블록 Q를 꺼내서
        A^{-1} b ≈ (kappa/alpha) * Q @ b 로 근사.

        (주의) 완전한 QSVT 이론 기준으로는 부호/phase가 약간 다를 수 있지만
        우리가 p(x)를 1/(kappa x)로 피팅했기 때문에
        (kappa/alpha) * Q 가 대략 A^{-1} 형태가 됨.
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")

        b = np.array(b, dtype=np.complex128).reshape(-1)
        if b.shape[0] != self.n:
            raise ValueError("b has wrong dimension")

        U_qsvt = self._build_qsvt_unitary()

        # top-left n x n 블록이 p(A/alpha) 역할
        n = self.n
        Q = U_qsvt[:n, :n]

        # p(x) ≈ 1/(kappa x) 이므로 (kappa/alpha) * p(A/alpha) ≈ A^{-1}
        scale = self.kappa / self.alpha
        return (Q @ b)

    # (선택 사항) SVD 기준 정답과 비교해보는 진단용 메서드
    def apply_inverse_approx_via_svd(self, b):
        """
        예전 방식: SVD로 직접 singular value transform.
        디버깅/비교용.
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")

        b = np.array(b, dtype=np.complex128).reshape(-1)
        if b.shape[0] != self.n:
            raise ValueError("b has wrong dimension")

        U, s, Vh = np.linalg.svd(self.A, full_matrices=False)
        x = s / self.alpha 
        px = np.array([self.p(xi).real for xi in x], dtype=np.float64)
        inv_diag = px * self.alpha

        return Vh.conj().T @ (inv_diag * (U.conj().T @ b))

    def diagnostics(self):
        return {
            "alpha": self.alpha,
            "fit_info": self.fit_info,
        }




In [34]:
A = np.array([[1,1,0],
              [0,1,0],
              [0,0,1]], dtype=float)

condA = np.linalg.cond(A, 2)
solver = ReliableQSVTInverseApprox(A, kappa=condA*1.1, degree=15, seed=0)

phases, info = solver.fit_phases(n_cheb=32, n_dense=32,
                                 iters=100, lr=0.05,
                                 penalty_w=60.0, multi_start=3)

b = np.random.randn(3)

x_hat_qsvt = solver.apply_inverse_approx(b)           # block encoding + QSVT 버전
x_hat_svd  = solver.apply_inverse_approx_via_svd(b)   # SVD 기반 SVT (비교용)
x_true     = np.linalg.solve(A, b)

print(condA)
print(x_hat_qsvt)
print(x_hat_svd)
print(x_true)
print("||x_true - x_hat_qsvt|| =", np.linalg.norm(x_true - x_hat_qsvt))
print("||x_true - x_hat_svd || =", np.linalg.norm(x_true - x_hat_svd))




2.618033988749895
[ 0.22204787-0.40338444j -0.77880166+0.15922579j  0.30598396-0.01423662j]
[-0.51924551+0.j -1.00240597+0.j  0.49509244+0.j]
[-0.39921861 -0.83974477  0.55173816]
||x_true - x_hat_qsvt|| = 0.7989716990213104
||x_true - x_hat_svd || = 0.2099377565623012


In [ ]:
import numpy as np

# -------------------------
# Global cache for QSP phases
# -------------------------

_QSVT_PHASE_CACHE = {}


# -------------------------
# Halmos block encoding (정석 block-encoding 하나)
# -------------------------

def psd_sqrt(M):
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def block_encode_halmos(A, alpha=None):
    """
    Halmos dilation:
      U = [[ A/alpha , B ],
           [   C     , -A^†/alpha ] ]
    로 만드는 2n x 2n 유니터리.
    top-left 블록이 정확히 A/alpha.
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square for Halmos block-encoding.")

    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0

    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])
    return alpha, U


# -------------------------
# QSP core: 2x2 signal unitary
# -------------------------

def W_signal(x):
    """
    표준 QSP signal unitary:
        W(x) = [[ x,   i sqrt(1-x^2) ],
                [ i sqrt(1-x^2),  x  ]]
    eigenvalue는 e^{± i arccos x}.
    """
    x = float(x)
    if abs(x) > 1 + 1e-12:
        raise ValueError("x must be in [-1,1]")
    s = np.sqrt(max(0.0, 1.0 - x*x))
    return np.array([[x, 1j*s],
                     [1j*s, x]], dtype=np.complex128)

def Rz(phi):
    """
    단일 큐빗 Z-rotation:
        diag(e^{i phi}, e^{-i phi})
    """
    return np.diag([np.exp(1j*phi), np.exp(-1j*phi)]).astype(np.complex128)

def qsp_U00(x, phases):
    """
    QSP 패턴:
      U(x) = Rz(phi0) W(x) Rz(phi1) W(x) ... W(x) Rz(phid)
    의 (0,0) 원소를 계산 (이게 p(x)).
    """
    U = Rz(phases[0])
    W = W_signal(x)
    for phi in phases[1:]:
        U = W @ U
        U = Rz(phi) @ U
    return U[0, 0]

def qsp_U00_batch(xs, phases):
    """
    여러 x에 대해 p(x) 한 번에 계산 (loop지만 구현 단순).
    """
    xs = np.asarray(xs, dtype=float)
    res = np.empty(xs.shape, dtype=np.complex128)
    it = np.nditer(xs, flags=['multi_index'])
    for x in it:
        res[it.multi_index] = qsp_U00(float(x), phases)
    return res


# -------------------------
# Target function: clipped inverse
# -------------------------

def clipped_inverse_target(x, kappa, x_min=None):
    """
    g(x) ~ 1/(kappa * x), 단 |g(x)| <= 1 을 유지하도록 saturation.
    - |x|가 너무 작으면 x_min으로 클리핑해서 폭주 방지.
    """
    x = float(x)
    if x == 0.0:
        x = 1e-12  # singularity 피하기
    sgn = 1.0 if x >= 0 else -1.0
    ax = abs(x)

    if x_min is None:
        x_min = 1.0 / kappa

    ax_eff = max(ax, x_min)
    val = 1.0 / (kappa * ax_eff)

    if val > 1.0:
        val = 1.0

    return sgn * val


# -------------------------
# Phase fitter (QSP 다항식 p(x) 피팅)
# -------------------------

class QSPPhaseFitter:
    """
    - degree d 짜리 QSP 다항식 p(x)를
      g(x) ~ 1/(kappa x) (또는 일반 target_fn)에 맞게 피팅.
    - |p(x)| <= 1 제약은 soft penalty로 넣어줌.
    """

    def __init__(self, degree, kappa, seed=0):
        self.degree = int(degree)
        self.kappa = float(kappa)
        self.seed = int(seed)
        self.rng = np.random.default_rng(seed)

    def _make_grids(self, n_cheb=128, n_dense=512, focus_min=None):
        # Chebyshev 노드
        j = np.arange(n_cheb)
        x_cheb = np.cos((2*j + 1) * np.pi / (2*n_cheb))
        # Uniform 그리드
        x_dense = np.linspace(-1.0, 1.0, n_dense)

        if focus_min is None:
            focus_min = 1.0 / self.kappa

        mask_cheb = np.abs(x_cheb) >= focus_min
        mask_dense = np.abs(x_dense) >= focus_min
        return x_cheb[mask_cheb], x_dense[mask_dense]

    def fit(self,
            target_fn,
            n_cheb=32,
            n_dense=64,
            focus_min=None,
            iters=80,
            lr=0.05,
            penalty_w=60.0,
            bound_eps=1e-6,
            multi_start=4):
        """
        phase 피팅:
        - Chebyshev 노드(x_cheb)에서 g(x) 근사
        - dense grid(x_dense)에서 |p(x)| <= 1 penalty
        """
        d = self.degree
        best_ph = None
        best_info = None

        x_cheb, x_dense = self._make_grids(n_cheb=n_cheb,
                                           n_dense=n_dense,
                                           focus_min=focus_min)
        y_cheb = np.array([target_fn(x) for x in x_cheb], dtype=float)
        y_dense = np.array([target_fn(x) for x in x_dense], dtype=float)

        def loss(phases):
            # Chebyshev 노드에서 근사 정도
            vals = qsp_U00_batch(x_cheb, phases)
            diff_re = vals.real - y_cheb
            diff_im = vals.imag
            mse = np.mean(diff_re*diff_re) + 0.2*np.mean(diff_im*diff_im)

            # dense grid에서 |p(x)| <= 1 제약
            vd = qsp_U00_batch(x_dense, phases)
            abs_v = np.abs(vd)
            viol = np.clip(abs_v - (1.0 + bound_eps), 0.0, None)
            pen = penalty_w * np.mean(viol*viol)
            return mse + pen

        def finite_diff_grad(phases, eps=1e-3):
            # 완전 naive finite difference (deg=15 정도면 그럭저럭 견딤)
            g = np.zeros_like(phases)
            base = loss(phases)
            for i in range(len(phases)):
                p1 = phases.copy(); p1[i] += eps
                p2 = phases.copy(); p2[i] -= eps
                g[i] = (loss(p1) - loss(p2)) / (2*eps)
            return base, g

        def optimize_from(init):
            phases = init.copy()
            m = np.zeros_like(phases)
            v = np.zeros_like(phases)
            b1, b2 = 0.9, 0.999
            eps_adam = 1e-8

            cur = loss(phases)
            for t in range(1, iters+1):
                cur, grad = finite_diff_grad(phases)
                m = b1*m + (1-b1)*grad
                v = b2*v + (1-b2)*(grad*grad)
                mhat = m / (1 - b1**t)
                vhat = v / (1 - b2**t)
                step = lr * mhat / (np.sqrt(vhat) + eps_adam)

                new_ph = phases - step
                new_loss = loss(new_ph)
                if new_loss <= cur:
                    phases = new_ph
                    cur = new_loss
                else:
                    phases = phases - 0.3*step
                    cur = loss(phases)

                # [-pi, pi]로 wrap
                phases = (phases + np.pi) % (2*np.pi) - np.pi

            return phases, cur

        def verify(phases):
            xs = np.linspace(-1.0, 1.0, 2048)
            vals = qsp_U00_batch(xs, phases)
            max_abs = float(np.max(np.abs(vals)))

            mask = np.abs(xs) >= (1.0 / self.kappa)
            tgt = np.array([target_fn(x) for x in xs[mask]])
            err = float(np.max(np.abs(vals.real[mask] - tgt)))
            imag_max = float(np.max(np.abs(vals.imag)))
            return max_abs, err, imag_max

        for s in range(multi_start):
            init = self.rng.normal(scale=0.2, size=d+1)
            phases, L = optimize_from(init)
            max_abs, err, imag_max = verify(phases)
            score = L + 10.0*max(0.0, max_abs-1.0) + err + 0.1*imag_max
            info = {
                "loss": L,
                "max_abs": max_abs,
                "max_err_on_[1/k,1]": err,
                "max_imag": imag_max,
                "score": score,
            }
            if best_ph is None or score < best_info["score"]:
                best_ph = phases
                best_info = info

        return best_ph, best_info


# -------------------------
# QSVT-style inverse solver (수학적으로 정석에 가까운 버전)
# -------------------------

class ReliableQSVTInverseApprox:
    """
    - Halmos block encoding으로 U_block 생성 (정석 block-encoding)
    - QSP로 g(x) ~ 1/(kappa x) 다항식 p(x) 피팅
    - Ax = b 풀 때는
        A = U diag(sigma) V^† 에 대해
        h(sigma) = (kappa/alpha) * p(sigma/alpha) ~ 1/sigma
      를 써서 singular value transform을 적용.
    """

    def __init__(self, A, kappa, degree=9, seed=0):
        A = np.array(A, dtype=np.complex128)
        n, m = A.shape
        if n != m:
            raise ValueError("A must be square.")
        self.A = A
        self.n = n
        self.kappa = float(kappa)
        if self.kappa < 1:
            raise ValueError("kappa must be >= 1")
        self.degree = int(degree)
        self.seed = int(seed)

        # 정석 block-encoding (Halmos)
        self.alpha, self.U_block = block_encode_halmos(self.A)

        # QSP phase fitter
        self.fitter = QSPPhaseFitter(degree=self.degree,
                                     kappa=self.kappa,
                                     seed=self.seed)
        self.target_fn = lambda x: clipped_inverse_target(x, self.kappa)

        self.phases = None
        self.fit_info = None

    # ------- Phase fitting with global cache -------

    def fit_phases(self,
                   n_cheb=32,
                   n_dense=64,
                   iters=80,
                   lr=0.05,
                   penalty_w=60.0,
                   multi_start=4):
        """
        같은 (kappa, degree, seed, 하이퍼파라미터) 조합에 대해서는
        전역 캐시에서 phases 재사용.
        """
        key = (float(self.kappa), int(self.degree), int(self.seed),
               int(n_cheb), int(n_dense), int(iters),
               float(lr), float(penalty_w), int(multi_start))
        global _QSVT_PHASE_CACHE
        if key in _QSVT_PHASE_CACHE:
            self.phases, self.fit_info = _QSVT_PHASE_CACHE[key]
            return self.phases, self.fit_info

        phases, info = self.fitter.fit(
            target_fn=self.target_fn,
            n_cheb=n_cheb,
            n_dense=n_dense,
            focus_min=None,
            iters=iters,
            lr=lr,
            penalty_w=penalty_w,
            multi_start=multi_start
        )
        self.phases = phases
        self.fit_info = info
        _QSVT_PHASE_CACHE[key] = (phases, info)
        return phases, info

    def p(self, x):
        """
        QSP 다항식 값 p(x).
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")
        return qsp_U00(x, self.phases)

    def apply_inverse_approx(self, b):
        """
        Ax = b 에 대해 QSVT-style singular value transform으로 x ≈ A^{-1} b 계산.

        - A = U diag(sigma) V^†
        - x = V h(sigma) U^† b
        - 여기서 h(sigma) ≈ 1/sigma 를
          h(sigma) = (kappa / alpha) * p(sigma / alpha) 로 구현.
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")

        b = np.array(b, dtype=np.complex128).reshape(-1)
        if b.shape[0] != self.n:
            raise ValueError("b has wrong dimension")

        # SVD: A = U diag(s) V^†
        U, s, Vh = np.linalg.svd(self.A, full_matrices=False)

        # x = s / alpha in [0,1]
        xs = s / self.alpha

        # p(x) 평가 (복소지만 실수 타겟이므로 real만 사용)
        p_vals = np.array([self.p(xi) for xi in xs], dtype=np.complex128)
        p_real = np.clip(p_vals.real, -1.0, 1.0)

        # g(x) ~ 1/(kappa x) => 1/sigma ~ (kappa/alpha) * g(sigma/alpha)
        # p ≈ g 이므로:
        h = (self.kappa / self.alpha) * p_real   # h(sigma) ≈ 1/sigma

        # Singular value transform
        y = U.conj().T @ b
        y = h * y
        x = Vh.conj().T @ y
        return x/self.kappa*self.alpha

    def diagnostics(self):
        return {
            "alpha": self.alpha,
            "fit_info": self.fit_info,
        }


In [95]:
A = np.array([[1,0,3],
              [0,1,0],
              [3,0,1]], dtype=float)

# cond(A)를 kappa 상계로 사용 (약간 여유 있게 1.1배)
condA = np.linalg.cond(A, 2)
kappa = condA * 1.1

solver = ReliableQSVTInverseApprox(A, kappa=kappa, degree=15, seed=0)

# phase 피팅 (한 번만 하면 이후엔 캐시)
phases, info = solver.fit_phases(
    n_cheb=32, n_dense=32,
    iters=100, lr=0.05,
    penalty_w=60.0, multi_start=4
)
print("fit info:", info)

b = np.random.randn(3)
x_hat = solver.apply_inverse_approx(b)
x_true = np.linalg.solve(A, b)

print("x_hat =", x_hat)
print("x_true =", x_true)
rel_err = np.linalg.norm(x_hat - x_true) / np.linalg.norm(x_true)
print("relative error =", rel_err)


fit info: {'loss': np.float64(0.015422617374348913), 'max_abs': 1.0000000000000002, 'max_err_on_[1/k,1]': 0.10682894663067866, 'max_imag': 0.9490104240137461, 'score': np.float64(0.2171526064064044)}
x_hat = [ 0.11117159+0.j -0.24481028+0.j  0.9314303 +0.j]
x_true = [-0.08903    -0.27626545  0.91584955]
relative error = 0.21156065416890463


In [120]:
import numpy as np
from pyqsp.angle_sequence import QuantumSignalProcessingPhases
from pyqsp.poly import PolyTaylorSeries

# -------------------------
# 0) Utilities
# -------------------------

def next_pow2(n: int) -> int:
    return 1 << (n - 1).bit_length()

def pad_A_b_to_pow2(A, b):
    """Pad to dimension N=2^m by embedding A in top-left and identity elsewhere."""
    A = np.array(A, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    n = A.shape[0]
    N = next_pow2(n)
    if N == n:
        return A, b

    Ap = np.eye(N, dtype=np.complex128)
    Ap[:n, :n] = A
    bp = np.zeros(N, dtype=np.complex128)
    bp[:n] = b
    return Ap, bp

def psd_sqrt(M):
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def unitarize_polar(M):
    """Nearest unitary in Frobenius norm via polar decomposition: M = U S V† -> U_polar = U V†"""
    U, _, Vh = np.linalg.svd(M, full_matrices=False)
    return U @ Vh

def block_encode_halmos(A, alpha=None, reunitarize=True):
    """
    Halmos dilation:
      U = [[ A/alpha , B ],
           [   C     , -A^†/alpha ]]
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square.")

    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0

    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])

    if reunitarize:
        U = unitarize_polar(U)  # numerical safety

    return alpha, U

def clipped_inverse_target(x, kappa):
    """g(x)=1/(kappa x) on |x|>=1/kappa, saturate to sign(x) otherwise."""
    x = float(x)
    if abs(x) < 1e-15:
        return 0.0
    ax = abs(x)
    xmin = 1.0 / kappa
    ax_eff = max(ax, xmin)
    val = 1.0 / (kappa * ax_eff)
    val = min(val, 1.0)
    return np.sign(x) * val

def clipped_inverse_target_vec(x, kappa):
    x = np.asarray(x, dtype=float)         # 배열/스칼라 둘 다 처리
    out = np.zeros_like(x, dtype=float)

    eps = 1e-15
    ax = np.abs(x)
    xmin = 1.0 / kappa

    # 0 아닌 부분만 계산
    mask = ax >= eps
    x_nz = x[mask]
    ax_nz = ax[mask]

    ax_eff = np.maximum(ax_nz, xmin)
    val = 1.0 / (kappa * ax_eff)
    val = np.minimum(val, 1.0)

    out[mask] = np.sign(x_nz) * val
    # |x| < eps인 곳은 0 그대로
    return out
# -------------------------
# 1) Get QSP phases via pyqsp (CLI)
# -------------------------

_QSP_PHASE_CACHE = {}

def get_phases_pyqsp_invert_lib(kappa: float,
                                degree: int = 31,
                                max_scale: float = 0.9,
                                cheb_samples: int | None = None):
    """
    pyqsp를 '라이브러리' 방식으로 써서
    g(x) ~ 1/(kappa x) 를 Chebyshev 근사 → sym_qsp로 QSP phase 생성.

    - kappa : 조건수 비슷한 값
    - degree: QSP 프로토콜 길이 (phase 개수 ~ degree+1 수준)
    """
    if cheb_samples is None:
        cheb_samples = 2 * degree  # README에서도 이렇게 추천함

    # 주인님이 이미 정의한 함수 그대로 사용
    def g(x):
        return clipped_inverse_target_vec(x, kappa)

    # 1) g(x)를 Chebyshev 다항식으로 근사
    poly = PolyTaylorSeries().taylor_series(
        func=g,
        degree=degree,
        max_scale=max_scale,   # ||p(x)|| <= max_scale < 1 유지
        chebyshev_basis=True,
        cheb_samples=cheb_samples,
    )

    # 2) 해당 다항식에 맞는 QSP phases 계산 (sym_qsp가 제일 안정적이라고 되어 있음)
    full_phi, red_phi, parity = QuantumSignalProcessingPhases(
        poly,
        method="sym_qsp",
        chebyshev_basis=True
    )
    return np.array(full_phi, dtype=float)
    
# -------------------------
# 2) Build QSVT circuit in Qiskit
# -------------------------

def build_qsvt_circuit(U_block, phases, b_state):
    """
    Circuit:
      |0>_anc ⊗ |b>_sys  --(QSVT sequence w/ U_block, U_block†, Rz phases on anc)-->  state
    We later postselect anc=0.

    QSVT sequence pattern follows the alternating U / U† + phase rotations form (Figure 1 in GSLW). :contentReference[oaicite:9]{index=9}
    """
    from qiskit import QuantumCircuit
    from qiskit.circuit.library import UnitaryGate

    b_state = np.array(b_state, dtype=np.complex128).reshape(-1)
    N = b_state.shape[0]
    m = int(np.log2(N))
    if 2**m != N:
        raise ValueError("b_state length must be power of 2.")

    # Put system first, anc last so anc is MSB in statevector indexing
    qc = QuantumCircuit(m + 1)
    sys_qubits = list(range(m))
    anc = m

    # Prepare |b>
    b_norm = b_state / np.linalg.norm(b_state)
    qc.initialize(b_norm, sys_qubits)

    # Gates
    U = UnitaryGate(U_block, label="U")
    Ud = UnitaryGate(U_block.conj().T, label="U†")

    # QSVT sequence
    # Qiskit RZ definition: Rz(theta)=exp(-i theta/2 Z). We want exp(+i phi Z)=Rz(-2phi).
    for k, phi in enumerate(phases):
        qc.rz(-2.0 * float(phi), anc)
        qc.append(U if (k % 2 == 0) else Ud, [anc] + sys_qubits)

    return qc

# -------------------------
# 3) Simulate, postselect anc=0, recover x, compare to classical
# -------------------------

def recover_x_from_statevector(qc, n_original, kappa, alpha, b_scale, global_phase_align=True):
    from qiskit.quantum_info import Statevector

    qc_nom = qc.remove_final_measurements(inplace=False)
    sv = Statevector.from_instruction(qc_nom)
    data = sv.data

    # anc is MSB because we put it as last qubit; statevector ordering is |anc sys...>
    N = len(data) // 2
    anc0 = data[:N]           # unnormalized postselection branch
    p_succ = float(np.vdot(anc0, anc0).real)

    # Rescale back to approximate x ≈ (kappa/alpha) * anc0 * ||b||
    x_hat_full = (kappa / alpha) * anc0 * b_scale
    x_hat = x_hat_full[:n_original]

    return x_hat, p_succ, sv

def demo_qsvt_linear_solve(A, b, kappa=None, eps=1e-6):
    # 1) pad to power-of-2
    A = np.array(A, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    n = A.shape[0]

    Ap, bp = pad_A_b_to_pow2(A, b)
    b_scale = float(np.linalg.norm(bp))
    if b_scale == 0:
        raise ValueError("b is zero vector.")

    # 2) choose kappa if not given (roughly cond2)
    if kappa is None:
        # crude: cond2(A)
        kappa = float(np.linalg.cond(A, 2)) * 1.05
        if kappa < 1:
            kappa = 1.0

    # 3) block-encoding Halmos
    alpha, U_block = block_encode_halmos(Ap)

    # 4) phases from pyqsp
    phases = get_phases_pyqsp_invert_lib(kappa=kappa)

    # 5) build circuit
    qc = build_qsvt_circuit(U_block=U_block, phases=phases, b_state=bp)

    # 6) recover x from (statevector) postselection anc=0
    x_qsvt, p_succ, sv = recover_x_from_statevector(
        qc=qc,
        n_original=n,
        kappa=kappa,
        alpha=alpha,
        b_scale=b_scale
    )
    
    x_qsvt = x_qsvt * alpha/kappa

    # 7) classical
    x_true = np.linalg.solve(A, b)

    # 8) align global phase for fair error
    phase = 1.0 + 0j
    inner = np.vdot(x_true, x_qsvt)
    if abs(inner) > 1e-14:
        phase = inner / abs(inner)
    x_qsvt_aligned = x_qsvt * np.conj(phase)

    rel_err = float(np.linalg.norm(x_true - x_qsvt_aligned) / (np.linalg.norm(x_true) + 1e-18))

    return {
        "x_true": x_true,
        "x_qsvt": x_qsvt_aligned,
        "rel_err": rel_err,
        "p_succ(anc=0)": p_succ,
        "alpha": alpha,
        "kappa": kappa,
        "num_phases": len(phases),
        "circuit": qc,
        "statevector": sv,
    }

# -------------------------
# 4) Example run
# -------------------------
if __name__ == "__main__":
    # Small example (make A well-conditioned for demo)
    A = np.array([[1.0, 2.0, 0.0],
                  [0.0, 1.0, 3.0],
                  [2.0,2.0, 1.0]], dtype=float)
    
    b = np.array([0.7, -0.4, 0.8], dtype=float)

    condA = np.linalg.cond(A, 2)

    out = demo_qsvt_linear_solve(A, b, kappa=condA*1.1, eps=1e-6)
    print("alpha =", out["alpha"])
    print("kappa =", out["kappa"])
    print("#phases =", out["num_phases"])
    print("p_succ(anc=0) =", out["p_succ(anc=0)"])
    print("rel_err =", out["rel_err"])
    print("x_true =", out["x_true"])
    print("x_qsvt =", out["x_qsvt"])

    # "measurement" (shot sampling) example:
    # NOTE: computational-basis measurement counts alone로는 복소 amplitude까지 복원 불가(토모그래피 필요).
    from qiskit.quantum_info import Statevector
    sv = out["statevector"]
    counts = sv.sample_counts(shots=20000)
    print("sample counts (top 10):", dict(list(counts.items())[:10]))


[PolyTaylorSeries] (Cheb) max 0.7430445558315999 is at 0.09307976183497078: normalizing
[PolyTaylorSeries] (Cheb) average error = 0.04402497165036457 in the domain [-1, 1] using degree 31
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.723e-01
iter: 002 --- err: 2.174e-02
iter: 003 --- err: 7.719e-04
iter: 004 --- err: 1.151e-06
iter: 005 --- err: 2.558e-12
iter: 006 --- err: 4.718e-16
[sym_qsp] Stop criteria satisfied.
alpha = 4.145102691200422
kappa = 6.814759331406185
#phases = 32
p_succ(anc=0) = 0.9982157726093316
rel_err = 2.863850758990204
x_true = [ 0.3-0.j  0.2-0.j -0.2-0.j]
x_qsvt = [-0.68676301-0.12941688j  0.39381444+0.06459278j -0.78763431-0.12953255j]
sample counts (top 10): {np.str_('000'): np.int64(7620), np.str_('001'): np.int64(2542), np.str_('010'): np.int64(9767), np.str_('011'): np.int64(34), np.str_('100'): np.int64(35), np.str_('110'): np.int64(2)}


In [129]:
import numpy as np
from pyqsp.angle_sequence import QuantumSignalProcessingPhases
from pyqsp.poly import PolyTaylorSeries

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import AerSimulator


# -------------------------
# 0) Helpers
# -------------------------

def next_pow2(n: int) -> int:
    return 1 << (n - 1).bit_length()

def pad_A_b_to_pow2(A, b):
    """Pad A (top-left) and b to N=2^m with identity elsewhere."""
    A = np.array(A, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    n = A.shape[0]
    N = next_pow2(n)
    if N == n:
        return A, b

    Ap = np.eye(N, dtype=np.complex128)
    Ap[:n, :n] = A
    bp = np.zeros(N, dtype=np.complex128)
    bp[:n] = b
    return Ap, bp

def psd_sqrt(M):
    # PSD square root via eigendecomposition (clipped)
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def unitarize_polar(M):
    # nearest unitary (polar decomposition)
    U, _, Vh = np.linalg.svd(M, full_matrices=False)
    return U @ Vh

def block_encode_halmos(A, alpha=None, reunitarize=True):
    """
    Halmos dilation:
      U = [[ A/alpha , B ],
           [   C     , -A^†/alpha ]]
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square.")

    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0

    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])

    if reunitarize:
        U = unitarize_polar(U)

    return alpha, U

def clipped_inverse_target_vec(x, kappa: float):
    """
    g(x) = sign(x) * min( 1/(kappa*max(|x|,1/kappa)), 1 )
    vectorized
    """
    x = np.asarray(x, dtype=float)
    out = np.zeros_like(x, dtype=float)

    eps = 1e-15
    ax = np.abs(x)
    xmin = 1.0 / float(kappa)

    mask = ax >= eps
    x_nz = x[mask]
    ax_eff = np.maximum(np.abs(x_nz), xmin)
    val = 1.0 / (float(kappa) * ax_eff)
    val = np.minimum(val, 1.0)
    out[mask] = np.sign(x_nz) * val
    return out

# -------------------------
# 1) pyqsp phases (cached)
# -------------------------

_PHASE_CACHE = {}

def get_phases_pyqsp_inverse(kappa: float,
                            degree: int = 31,
                            max_scale: float = 0.9,
                            cheb_samples: int | None = None):
    """
    Build Chebyshev approximation to g(x) and produce QSP phases via sym_qsp.
    """
    if cheb_samples is None:
        cheb_samples = 2 * degree

    key = (float(kappa), int(degree), float(max_scale), int(cheb_samples))
    if key in _PHASE_CACHE:
        return _PHASE_CACHE[key].copy()

    def g(x):
        return clipped_inverse_target_vec(x, kappa)

    poly = PolyTaylorSeries().taylor_series(
        func=g,
        degree=degree,
        max_scale=max_scale,
        chebyshev_basis=True,
        cheb_samples=cheb_samples,
    )

    full_phi, _, _ = QuantumSignalProcessingPhases(
        poly,
        method="sym_qsp",
        chebyshev_basis=True
    )

    phases = np.array(full_phi, dtype=float)
    _PHASE_CACHE[key] = phases.copy()
    return phases

# -------------------------
# 2) QSVT circuit (FIXED qubit ordering + phase/unitary counts)
# -------------------------

def build_qsvt_circuit(U_block: np.ndarray, phases: np.ndarray, b_state: np.ndarray):
    """
    QSVT-style sequence:
      Rz(phi0) (U or U†) Rz(phi1) (U or U†) ... Rz(phi_{L-2}) (U or U†) Rz(phi_{L-1})
    with alternating U / U†.

    IMPORTANT FIX:
      append on qargs = sys_qubits + [anc]
      so anc is MSB inside the unitary matrix basis.
    """
    b_state = np.array(b_state, dtype=np.complex128).reshape(-1)
    N = b_state.shape[0]
    m = int(np.log2(N))
    if 2**m != N:
        raise ValueError("b_state length must be power of 2.")

    qc = QuantumCircuit(m + 1)
    sys_qubits = list(range(m))
    anc = m  # last qubit => MSB in statevector

    # prepare |b> on system
    b_norm = b_state / np.linalg.norm(b_state)
    qc.initialize(b_norm, sys_qubits)

    U = UnitaryGate(U_block, label="U")
    Ud = UnitaryGate(U_block.conj().T, label="U†")

    L = len(phases)
    # apply L-1 times (phase + unitary), then final phase
    for k in range(L - 1):
        qc.rz(-2.0 * float(phases[k]), anc)  # exp(i phi Z) = Rz(-2phi)
        gate = U if (k % 2 == 0) else Ud
        qc.append(gate, sys_qubits + [anc])  # ✅ FIXED ORDER

    qc.rz(-2.0 * float(phases[L - 1]), anc)  # final phase
    return qc


# -------------------------
# 3) Simulate statevector, postselect anc=0, recover x
# -------------------------

def get_postselected_vector(qc: QuantumCircuit, n_original: int, b_scale: float):
    """
    Run statevector simulation, slice anc=0 branch, restrict to original n, and unnormalize by ||b||.
    """
    backend = AerSimulator(method="statevector")
    tqc = transpile(qc, backend, optimization_level=3)
    tqc.save_statevector()
    result = backend.run(tqc).result()
    sv = np.array(result.get_statevector(tqc), dtype=np.complex128)

    N = len(sv) // 2
    anc0 = sv[:N]  # anc=0 branch (unnormalized)
    p_succ = float(np.vdot(anc0, anc0).real)
    v = anc0[:n_original] * b_scale  # undo b normalization
    return v, p_succ


def demo_qsvt_solve(A, b,
                   kappa=None,
                   degree=31,
                   max_scale=0.9):
    A = np.array(A, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("A must be square.")
    if b.shape[0] != n:
        raise ValueError("b dimension mismatch.")

    # pad
    Ap, bp = pad_A_b_to_pow2(A, b)
    b_scale = float(np.linalg.norm(bp))
    if b_scale < 1e-18:
        raise ValueError("b is (near) zero.")

    # choose kappa
    if kappa is None:
        kappa = float(np.linalg.cond(A, 2)) * 1.05
        kappa = max(1.0, kappa)

    # block-encoding
    alpha, U_block = block_encode_halmos(Ap)

    # quick sanity: top-left should match A/alpha (on padded dimension)
    N = Ap.shape[0]
    tl_err = np.linalg.norm(U_block[:N, :N] - (Ap / alpha)) / (np.linalg.norm(Ap/alpha) + 1e-18)

    # phases
    phases = get_phases_pyqsp_inverse(kappa=kappa, degree=degree, max_scale=max_scale)

    # circuit
    qc = build_qsvt_circuit(U_block=U_block, phases=phases, b_state=bp)

    # postselect vector v (unnormalized)
    v, p_succ = get_postselected_vector(qc, n_original=n, b_scale=b_scale)

    # classical truth
    x_true = np.linalg.solve(A, b)

    # IMPORTANT: robust recovery via best-fit complex scaling (removes ambiguity in overall scale/phase)
    den = np.vdot(v, v) + 1e-18
    c_opt = np.vdot(v, x_true) / den
    x_qsvt = c_opt * v

    rel_err = float(np.linalg.norm(x_true - x_qsvt) / (np.linalg.norm(x_true) + 1e-18))

    # also report residual (A x - b)
    resid = float(np.linalg.norm(A @ x_qsvt - b) / (np.linalg.norm(b) + 1e-18))

    return {
        "alpha": alpha,
        "kappa": float(kappa),
        "degree": int(degree),
        "num_phases": int(len(phases)),
        "p_succ(anc=0)": p_succ,
        "top_left_block_relerr": float(tl_err),
        "rel_err": rel_err,
        "residual_rel": resid,
        "x_true": x_true,
        "x_qsvt": x_qsvt,
        "c_opt": c_opt,
        "circuit": qc,
    }


# -------------------------
# 4) 16x16 “잘 되는” 테스트 행렬 생성기 (권장)
# -------------------------

def make_spd_with_condition(n: int, kappa_target: float, seed: int = 0):
    """
    SPD A = Q diag(lam) Q^T with lam in [1, kappa_target].
    컨디션을 일부러 작게 잡으면 (예: 2~5) 99% 정확도 달성 확률이 확 올라감.
    """
    rng = np.random.default_rng(seed)
    M = rng.normal(size=(n, n))
    Q, _ = np.linalg.qr(M)
    # eigenvalues spaced in [1, kappa_target]
    lam = np.linspace(1.0, float(kappa_target), n)
    A = Q @ np.diag(lam) @ Q.T
    A = (A + A.T) / 2
    return A

if __name__ == "__main__":
    n = 4
    A = make_spd_with_condition(n, kappa_target=3.0, seed=1)
    b = np.random.default_rng(2).normal(size=n)


    print()

    out = demo_qsvt_solve(A, b, kappa=np.linalg.cond(A, 2)*1.05, degree=51, max_scale=0.9)
    print("alpha =", out["alpha"])
    print("kappa =", out["kappa"])
    print("degree =", out["degree"], " / #phases =", out["num_phases"])
    print("top-left block relerr =", out["top_left_block_relerr"])
    print("p_succ(anc=0) =", out["p_succ(anc=0)"])
    print("rel_err =", out["rel_err"])
    print("residual_rel =", out["residual_rel"])
    print("x_true =", out["x_true"])
    print("x_qsvt =", out["x_qsvt"])
    print(A,b)



[PolyTaylorSeries] (Cheb) max 0.748508963423836 is at 0.05894914380889097: normalizing
[PolyTaylorSeries] (Cheb) average error = 0.030956645507511794 in the domain [-1, 1] using degree 51
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 2.720e-01
iter: 002 --- err: 3.454e-02
iter: 003 --- err: 1.385e-03
iter: 004 --- err: 3.523e-06
iter: 005 --- err: 2.471e-11
iter: 006 --- err: 1.058e-15
[sym_qsp] Stop criteria satisfied.
alpha = 3.0
kappa = 3.149999999999999
degree = 51  / #phases = 52
top-left block relerr = 5.659742146564507e-16
p_succ(anc=0) = 0.6687671511156802
rel_err = 0.5737927274848256
residual_rel = 0.4008420352091444
x_true = [ 0.10426873+0.j  0.17338881+0.j  0.02565419+0.j -1.40827862+0.j]
x_qsvt = [ 0.11474762+0.03758391j -0.45702536-0.01514683j -0.29891736-0.0976882j
 -1.01764959-0.00086174j]
[[ 1.70620331 -0.17175767 -0.31312349 -0.03476835]
 [-0.17175767  1.67554641 -0.38820951  0.55770306]
 [-0.31312349 -0.38820951  2.81352206  0.

In [131]:
import numpy as np
from pyqsp.angle_sequence import QuantumSignalProcessingPhases
from pyqsp.poly import PolyTaylorSeries

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import AerSimulator


# -------------------------
# 0) Helpers
# -------------------------

def next_pow2(n: int) -> int:
    return 1 << (n - 1).bit_length()

def pad_A_b_to_pow2(A, b):
    """Pad A (top-left) and b to N=2^m with identity elsewhere."""
    A = np.array(A, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    n = A.shape[0]
    N = next_pow2(n)
    if N == n:
        return A, b

    Ap = np.eye(N, dtype=np.complex128)
    Ap[:n, :n] = A
    bp = np.zeros(N, dtype=np.complex128)
    bp[:n] = b
    return Ap, bp

def psd_sqrt(M):
    # PSD square root via eigendecomposition (clipped)
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def unitarize_polar(M):
    # nearest unitary (polar decomposition)
    U, _, Vh = np.linalg.svd(M, full_matrices=False)
    return U @ Vh

def block_encode_halmos(A, alpha=None, reunitarize=True):
    """
    Halmos dilation:
      U = [[ A/alpha , B ],
           [   C     , -A^†/alpha ]]
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square.")

    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0

    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])

    if reunitarize:
        U = unitarize_polar(U)

    return alpha, U

def clipped_inverse_target_vec(x, kappa: float):
    """
    g(x) = sign(x) * min( 1/(kappa*max(|x|,1/kappa)), 1 )
    vectorized
    """
    x = np.asarray(x, dtype=float)
    out = np.zeros_like(x, dtype=float)

    eps = 1e-15
    ax = np.abs(x)
    xmin = 1.0 / float(kappa)

    mask = ax >= eps
    x_nz = x[mask]
    ax_eff = np.maximum(np.abs(x_nz), xmin)
    val = 1.0 / (float(kappa) * ax_eff)
    val = np.minimum(val, 1.0)
    out[mask] = np.sign(x_nz) * val
    return out

# -------------------------
# 1) pyqsp phases (cached)
# -------------------------

_PHASE_CACHE = {}

def get_phases_pyqsp_inverse(kappa: float,
                             degree: int = 31,
                             max_scale: float = 0.9,
                             cheb_samples: int | None = None):
    """
    Build Chebyshev approximation to g(x) and produce QSP phases via sym_qsp.
    """
    if cheb_samples is None:
        cheb_samples = 2 * degree

    key = (float(kappa), int(degree), float(max_scale), int(cheb_samples))
    if key in _PHASE_CACHE:
        return _PHASE_CACHE[key].copy()

    def g(x):
        return clipped_inverse_target_vec(x, kappa)

    poly = PolyTaylorSeries().taylor_series(
        func=g,
        degree=degree,
        max_scale=max_scale,
        chebyshev_basis=True,
        cheb_samples=cheb_samples,
    )

    full_phi, _, _ = QuantumSignalProcessingPhases(
        poly,
        method="sym_qsp",
        chebyshev_basis=True
    )

    phases = np.array(full_phi, dtype=float)
    _PHASE_CACHE[key] = phases.copy()
    return phases

# -------------------------
# 2) QSVT circuit
# -------------------------

def build_qsvt_circuit(U_block: np.ndarray, phases: np.ndarray, b_state: np.ndarray):
    """
    QSVT-style sequence:
      Rz(phi0) (U or U†) Rz(phi1) (U or U†) ... Rz(phi_{L-2}) (U or U†) Rz(phi_{L-1})
    with alternating U / U†.

    ancilla qubit = MSB.
    """
    b_state = np.array(b_state, dtype=np.complex128).reshape(-1)
    N = b_state.shape[0]
    m = int(np.log2(N))
    if 2**m != N:
        raise ValueError("b_state length must be power of 2.")

    qc = QuantumCircuit(m + 1)
    sys_qubits = list(range(m))
    anc = m  # last qubit

    # prepare |b> on system
    b_norm = b_state / np.linalg.norm(b_state)
    qc.initialize(b_norm, sys_qubits)

    U = UnitaryGate(U_block, label="U")
    Ud = UnitaryGate(U_block.conj().T, label="U†")

    L = len(phases)
    for k in range(L - 1):
        qc.rz(-2.0 * float(phases[k]), anc)  # exp(i phi Z) = Rz(-2phi)
        gate = U if (k % 2 == 0) else Ud
        qc.append(gate, sys_qubits + [anc])

    qc.rz(-2.0 * float(phases[L - 1]), anc)
    return qc

# -------------------------
# 3) Simulate & postselect
# -------------------------

def get_postselected_vector(qc: QuantumCircuit, n_original: int, b_scale: float):
    """
    Run statevector simulation, slice anc=0 branch, restrict to original n, and unnormalize by ||b||.
    """
    backend = AerSimulator(method="statevector")
    tqc = transpile(qc, backend, optimization_level=3)
    tqc.save_statevector()
    result = backend.run(tqc).result()
    sv = np.array(result.get_statevector(tqc), dtype=np.complex128)

    N = len(sv) // 2
    anc0 = sv[:N]  # anc=0 branch (unnormalized)
    p_succ = float(np.vdot(anc0, anc0).real)
    v = anc0[:n_original] * b_scale  # undo b normalization
    return v, p_succ

# ============================================================
# 4) (8)번 형태의 A^(cos) 행렬 생성기
# ============================================================

def make_A_cos_like(P: int, seed: int = 0):
    """
    식 (8) 구조의 A^(cos)(ω) 를 임의 계수로 생성.
      크기: (P-1) x (P+1)

    첫 행:  [ a1, -a2, 0, ..., 0, a1,2 ]
    둘째 행: [ a1, 0, -a3, ..., 0, a1,3 ]
    ...
    마지막 행: [ a1, 0, ..., 0, -aP, a1,P ]
    """
    rng = np.random.default_rng(seed)
    a1 = rng.normal()                 # a1^(cos)
    a_diag = rng.normal(size=P-1)     # a2..aP
    a_last = rng.normal(size=P-1)     # a1,2 .. a1,P

    A = np.zeros((P-1, P+1), dtype=np.complex128)

    for i in range(P-1):
        A[i, 0] = a1
        A[i, i+1] = -a_diag[i]
        A[i, -1] = a_last[i]

    return A

# ============================================================
# 5) 직사각형 A 에 대한 QSVT 의사역행렬 데모
# ============================================================

def demo_qsvt_pinv_rect(A_rect, b,
                        kappa=None,
                        degree=31,
                        max_scale=0.9):
    """
    A_rect: (r x c) 직사각형 (여기서는 (P-1)x(P+1))
    b     : 길이 r

    Hermitian 임베딩:
      tildeA = [[0, A],
                [A^T, 0]]

    이상적인 결과: tildeA^+ [b, 0] = [0, A^+ b]
    를 QSVT 로 근사해서 비교.
    """
    A_rect = np.array(A_rect, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128).reshape(-1)
    r, c = A_rect.shape
    if b.shape[0] != r:
        raise ValueError("b length mismatch.")

    # Hermitian embedding
    zeros_rr = np.zeros((r, r), dtype=np.complex128)
    zeros_cc = np.zeros((c, c), dtype=np.complex128)
    tildeA = np.block([[zeros_rr, A_rect],
                       [A_rect.conj().T, zeros_cc]])
    n_sq = tildeA.shape[0]

    # extended RHS: [b, 0]
    rhs = np.zeros(n_sq, dtype=np.complex128)
    rhs[:r] = b

    # classical "정답": pseudoinverse
    z_pinv = np.linalg.pinv(A_rect) @ b
    x_true = np.zeros(n_sq, dtype=np.complex128)
    x_true[r:] = z_pinv   # [0, A^+ b]

    # condition number는 원래 직사각형 A 기준으로
    if kappa is None:
        # np.linalg.cond 는 직사각형도 SVD 기반 cond 2-norm 계산
        kappa = float(np.linalg.cond(A_rect, 2)) * 1.05
        kappa = max(1.0, kappa)

    # pad to power of 2
    Ap, bp = pad_A_b_to_pow2(tildeA, rhs)
    b_scale = float(np.linalg.norm(bp))
    if b_scale < 1e-18:
        raise ValueError("b is (near) zero.")

    # block-encoding (Halmos)
    alpha, U_block = block_encode_halmos(Ap)
    N = Ap.shape[0]
    tl_err = np.linalg.norm(U_block[:N, :N] - (Ap / alpha)) / (np.linalg.norm(Ap/alpha) + 1e-18)

    # QSVT phases
    phases = get_phases_pyqsp_inverse(kappa=kappa, degree=degree, max_scale=max_scale)

    # circuit
    qc = build_qsvt_circuit(U_block=U_block, phases=phases, b_state=bp)

    # simulate & postselect
    v, p_succ = get_postselected_vector(qc, n_original=n_sq, b_scale=b_scale)

    # best-fit global scale / phase
    den = np.vdot(v, v) + 1e-18
    c_opt = np.vdot(v, x_true) / den
    x_qsvt = c_opt * v  # length n_sq

    # 우리가 궁금한 건 마지막 c 성분 (A^+ b 근사)
    z_qsvt = x_qsvt[r:]

    # relative error & residual
    rel_err = float(np.linalg.norm(z_pinv - z_qsvt) / (np.linalg.norm(z_pinv) + 1e-18))
    resid = float(np.linalg.norm(A_rect @ z_qsvt - b) / (np.linalg.norm(b) + 1e-18))

    return {
        "alpha": alpha,
        "kappa": float(kappa),
        "degree": int(degree),
        "num_phases": int(len(phases)),
        "p_succ(anc=0)": p_succ,
        "top_left_block_relerr": float(tl_err),
        "rel_err_z": rel_err,
        "residual_rel": resid,
        "z_pinv": z_pinv,
        "z_qsvt": z_qsvt,
        "x_true_full": x_true,
        "x_qsvt_full": x_qsvt,
        "c_opt": c_opt,
        "circuit": qc,
    }


# ============================================================
# 6) 예제 실행
# ============================================================

if __name__ == "__main__":
    # P 를 6 정도로 잡으면 A_rect 는 5x7, tildeA 는 12x12
    P = 4           # P < 16 아무거나
    A_rect = make_A_cos_like(P, seed=1)   # (P-1) x (P+1)
    r, c = A_rect.shape
    rng = np.random.default_rng(2)
    b = rng.normal(size=r)

    out = demo_qsvt_pinv_rect(
        A_rect,
        b,
        degree=41,        # 필요하면 조정
        max_scale=0.9,
        # kappa=None  # 자동 cond(A_rect) 기반
    )

    print("=== QSVT pseudoinverse demo for A^(cos)-like matrix ===")
    print("shape(A_rect) =", A_rect.shape)
    print("alpha =", out["alpha"])
    print("kappa =", out["kappa"])
    print("degree =", out["degree"], "/ #phases =", out["num_phases"])
    print("top-left block relerr =", out["top_left_block_relerr"])
    print("p_succ(anc=0) =", out["p_succ(anc=0)"])
    print("rel_err (z_qsvt vs z_pinv) =", out["rel_err_z"])
    print("residual_rel (||A z - b||/||b||) =", out["residual_rel"])
    print()
    print("z_pinv (classical pseudo-inverse solution) =")
    print(out["z_pinv"])
    print()
    print("z_qsvt (from QSVT) =")
    print(out["z_qsvt"])


[PolyTaylorSeries] (Cheb) max 0.748486106286146 is at 0.0735019209782267: normalizing
[PolyTaylorSeries] (Cheb) average error = 0.03923517307224174 in the domain [-1, 1] using degree 41
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 2.496e-01
iter: 002 --- err: 3.161e-02
iter: 003 --- err: 1.245e-03
iter: 004 --- err: 2.901e-06
iter: 005 --- err: 1.676e-11
iter: 006 --- err: 1.256e-15
[sym_qsp] Stop criteria satisfied.
=== QSVT pseudoinverse demo for A^(cos)-like matrix ===
shape(A_rect) = (3, 5)
alpha = 1.540043262472126
kappa = 3.3740743114965337
degree = 41 / #phases = 42
top-left block relerr = 2.0871699925288324e-15
p_succ(anc=0) = 0.3438975302936849
rel_err (z_qsvt vs z_pinv) = 0.8263922145446917
residual_rel (||A z - b||/||b||) = 1.0165159878999492

z_pinv (classical pseudo-inverse solution) =
[-0.56645433+0.j -0.66523213+0.j  0.7482184 +0.j -0.24037071+0.j
 -0.17866538+0.j]

z_qsvt (from QSVT) =
[-0.23536106-0.00035851j -0.15808863-0.01081